# Uczenie ze wzmocnieniem

W dotychczasowych wykładach zajmowaliśmy się dwoma skrajnymi typami uczenia: uczenie z nadzorem i bez nadzoru. 
* W pierwszym typie ciąg uczący zawiera wektory wejściowe i pożądane wartości wyjściowe. Na podstawie takiego zbioru możliwe jest bezpośrednie policzenie błędu popełnionego przez algorytm uczący i wyliczenie konkretnych poprawek.
* Drugą skrajnością były algorytmy analizy skupień bazujące na korelacjach między wejściem i wyjściem oraz na strukturze danych wejściowych.

## Uczenie z nagrodą i karą
W tym wykładzie zajmiemy się sytuacją pośrednią. 
* Dla każdego wejścia nie znamy co prawda prawidłowej wartości wyjścia, 
* ale umiemy powiedzieć czy było ono dobre czy złe. 

_Przypomina to trochę uczenie psa aportowania. Nie da się psu wytłumaczyć jak ma to zrobić, jedynie w wypadku powodzenia pies dostaje pochwałę/nagrodę a w przypadku niepowodzenia nic._

Podobnie w uczeniu ze wzmocnieniem będziemy jedynie wiedzieli kiedy algorytm nagrodzić (będziemy do tego używać funkcji nagrody) a kiedy karać.

## Przypisanie kary do kroku
Specyficznym problemem jest tu przypisanie kary za błąd. 

* Zwykle w zadaniach których rozwiązania chcielibyśmy nauczyć maszynę występuje wiele etapów i nie jest do końca jasne, który z nich przyczynił się do ostatecznego błędu. Można tu jako przykład rozważyć grę w szachy. 
  * Jeśli po 60 posunięciach partia się kończy naszą przegraną to nie jest jasne, który z naszych ruchów był nieprawidłowy.

Algorytmy tego typu znajdują bardzo wiele praktycznych zastosowań np: algorytmy do gier strategicznych, sterowanie robotami, optymalizacja przesyłania danych w sieciach komórkowych, efektywne indeksowanie stron internetowych itp.

Zaczniemy od omówienia procesu decyzyjnego Markowa (ang. Markov decision process MDP), który dostarcza formalizmu do opisu uczenia ze wzmocnieniem.

## Proces decyzyjny Markowa

Aby mówić o procesie decyzyjnym Markowa musimy zdefiniować następujący zestaw (krotkę): $(S,A,\lbrace P_{sa}\rbrace ,\gamma ,R)$ gdzie:
* $S$ jest zbiorem możliwych stanów, w jakich może znajdować się rozważany układ.
* $A$ to zbiór możliwych akcji
* $P_{s,a}$ to rozkład prawdopodobieństwa przejścia z bieżącego stanu do każdego z możliwych stanów, tak, że $P_{s,a_{i}}(s^{\prime })$ określa prawdopodobieństwo z jakim układ znajdujący się w stanie $s$ po wyborze akcji $a_{i}$ znajdzie się w stanie $s^{\prime }$.
* $\gamma $ to współczynnik o wartości $[0,1)$ służący do rozdzielania kary/nagrody pomiędzy kroki
* $R: S\times A \mapsto \mathcal {R}$ to funkcja nagrody.

### Strategia
* Aby "uruchomić" MDP potrzebna jest <i>strategia</i>, czyli mechanizm wybierania akcji.
* Formalnie strategia $\pi : S \mapsto A$ to funkcja, która mapuje stany na akcje. 
  * Czyli jeśli znajdujemy się w stanie $s$ to akcje wybieramy tak: $ a= \pi (s)$.
* Działanie MDP można opisać następująco: 
  * Początkowo układ znajduje się w stanie $s_{0}$. 
  * Zgodnie ze strategią $\pi (s_{0})$ wybierana jest akcja $a_{0}$. 
  * W sposób losowy, zgodnie z rozkładem $P_{s_{0},a_{0}}$ akcja $a_{0}$ przenosi układ do stanu $s_{1}$.
  * Następnie strategia $\pi (s_{1})$ decyduje jaką akcję $a_{1}$ wykonamy i z prawdopodobieństwem $P_{s_{1},a_{1}}$ przejdziemy do stanu $s_{2}$, 
  * itd. 
  

Na obrazku wygląda to tak:

$s_0 \, ^{\underrightarrow{a_0} }\, s_1 \, ^{\underrightarrow{a_1} }\,  s_2 \, ^{\underrightarrow{a_2} }\,  s_3\, ^{\underrightarrow{a_3} }\,   \dots $

## Przykład
Aby te pojęcia nabrały znaczenia rozważmy konkretny przykład. Mamy robota, "żyjącego" w świecie złożonym z 12 pól takim jak na poniższym rysunku.
![](https://brain.fuw.edu.pl/edu/images/8/87/Swiat_robota.png)


* Zajmowanie przez robota pola o konkretnych współrzędnych to jego stan, np. jeśli robot znajduje się w lewym górnym rogu to jest w stanie $s = (x=1,y=3)$. 
* Jego zbiór akcji to przemieszczenie się o jedną pozycję w jednym z czterech kierunków: $A = \lbrace N,E,S,W\rbrace $. 
* Robot nie może wejść na czarne pole, ani przeniknąć przez ściany. Próba podjęcia takiej niemożliwej akcji skutkuje tym, że robot pozostaje w dotychczasowym stanie.
* Chcielibyśmy, aby robot uruchomiony w takim świecie dotarł do stanu $(4,3)$ i jak tylko to możliwe unikał stanu $(4,2)$. 
* Przebywanie w każdym ze stanów związane jest z funkcją nagrody, zaznaczoną kolorami. 
  * Dla ustalenia uwagi weźmy funkcję nagrody taką, że:
    * stan $(4,3)$ ma nagrodę $R((4,3)) = +1$, 
    * $R((4,2)) = -1$, 
    * pozostałe stany mają wartość $R = -0.02$. 
  * Widać, że możemy nadawać funkcji nagrody wartości dodatnie (za przebywanie systemu w stanach, które uznajemy za korzystne) lub ujemne (w stanach mniej korzystnych).
* Dodatkowo załóżmy, że dynamika naszego robota jest stochastyczna. Innymi słowy ruchy naszego robota nie są idealnie precyzyjne, 
  * np. jeśli dostanie polecenie $N$ to przemieści się do góry z prawdopodobieństwem powiedzmy 0.8, na lewo z prawdopodobieństwem 0.1 i na prawo z prawdopodobieństwem 0.1.

Przy takiej dynamice rozkład prawdopodobieństwa przejścia między stanami może wyglądać np. tak:

$P_{(3,1),N}(3,2) = 0.8$ (Notacja ta oznacza prawdopodobieństwo przejścia ze stanu (3,1) do (3,2) po wybraniu akcji $N$)

$P_{(3,1),N}(2,1) = 0.1$

$P_{(3,1),N}(4,1) = 0.1$

$P_{(3,1),N}(3,3) = 0$

$\vdots $

Rozważmy teraz przykładowe działanie naszego robota, przyjmując strategię taką jak na poniższym rysunku.



<table class="wikitable";style = font-size:200%>
	<tr>
		<td>E</td>
		<td>E</td>
		<td>E</td>
		<td>+1</td>
	</tr>
	<tr>
		<td>N</td>
		<td>•</td>
		<td>N</td>
		<td>-1</td>
	</tr>
	<tr>
		<td>N</td>
		<td>W</td>
		<td>W</td>
		<td>W</td>
	</tr>
</table>

* Umieszczamy go w chwili $t_{0}$ w jakimś stanie $s_{0}$ (tzn. na którymś konkretnym polu)

* Wybieramy akcję $a_{0}$

* W zależności od $s_{0}$ i $a_{0}$ robot znajdzie się w stanie $s_{1}$ (Stan $s_{1}$ jest losowany z rozkładu prawdopodobieństwa $P_{s_{0},a_{0}}$ ; możemy to zapisać $s_{1} \sim P_{s_{0},a_{0}}$)

* Wybieramy akcję $a_{1}$

* Losujemy stan $s_{2} \sim P_{s_{1},a_{1}}$

* $\dots $

### Zysk
Z przebyciem konkretnej sekwencji stanów związany jest całkowity zysk, który może być obliczony z funkcji nagrody w następujący sposób:

$$Z = R(s_{0}) + \gamma R(s_{1})+ \gamma ^{2} R(s_{2}) + \dots $$

* Widzimy, że współczynnik $\gamma $ odpowiadający krokowi $k$ występuje w potędze $k$. 

* Ponieważ jest on dodatni i mniejszy niż 1 oznacza to, że na wartość oczekiwaną zysku mają największy wpływ wartości funkcji nagrody w najwcześniejszych korkach. 

* Możemy o tym myśleć w analogii do pieniędzy: Dobrze ulokowane pieniądze przynoszą tym większy zysk im wcześniej je ulokowaliśmy.

### Cel uczenia ze wzmcnieniem
Naszym celem w uczeniu ze wzmocnieniem jest takie wybieranie akcji (znalezienie takiej strategii) aby zmaksymalizować wartość oczekiwaną całkowitego zysku:

$$E [Z] =E [R(s_{0}) + \gamma R(s_{1})+ \gamma ^{2} R(s_{2}) + \dots ]$$

## Jak wyznaczyć optymalną strategię?
### Funkcja wartościująca
Pora wprowadzić pojęcie funkcji wartościującej, która posłuży do oceniania strategii: 

**Funkcja wartościująca dla danej strategii $\pi $ jest to wartość oczekiwana całkowitego zysku dla przypadku gdy startujemy ze stanu $s_{0}$ i podejmujemy akcje zgodnie ze strategią $\pi $.**

Można to zapisać następująco:


$$
V^{\pi }(s) = E[R(s_{0}) + \gamma R(s_{1})+ \gamma ^{2} R(s_{2}) + \dots | s_{0}=s, \pi ]
$$

Dla przykładu policzmy tą funkcje dla konkretnej strategii:
Strategia:


<table class="wikitable";style = font-size:100%>
	<tr>
		<td>E</td>
		<td>E</td>
		<td>E</td>
		<td>+1</td>
	</tr>
	<tr>
		<td>S</td>
		<td>•</td>
		<td>E</td>
		<td>-1</td>
	</tr>
	<tr>
		<td>E</td>
		<td>E</td>
		<td>N</td>
		<td>N</td>
	</tr>
</table>

Funkcja wartościująca dla tej strategii:


<table class="wikitable";style = font-size:100%>
	<tr>
		<td>0.52</td>
		<td>0.73</td>
		<td>0.77</td>
		<td>+1</td>
	</tr>
	<tr>
		<td>-0.90</td>
		<td>•</td>
		<td>-0.82</td>
		<td>-1</td>
	</tr>
	<tr>
		<td>-0.88</td>
		<td>-0.87</td>
		<td>-0.85</td>
		<td>-1</td>
	</tr>
</table>

Równanie poprzednie można przepisać w postaci:

$\qquad$ $$V^{\pi }(s) = E[R(s_{0}) + \gamma \left( R(s_{1})+ \gamma ^{1} R(s_{2}) + \gamma ^{2} R(s_{3})+ \dots \right) | s_{0}=s, \pi ]$$

Ponieważ $R(s_{0})$ jest stałe to mamy:

$\qquad$ $$V^{\pi }(s) = R(s_{0}) + \gamma E[\left( R(s_{1})+ \gamma ^{1} R(s_{2}) + \gamma ^{2} R(s_{3})+ \dots \right) | s_{0}=s, \pi ]$$

Zauważmy, że wyrażenie: 

$\qquad$ $$ E[R(s_{1})+ \gamma ^{1} R(s_{2}) + \gamma ^{2} R(s_{3})+ \dots] $$

to funkcja wartościująca strategi dla stanu $s_{1}$: $V^{\pi }(s_{1})$.

Przechodząc do notacji: 

$s_{0} \rightarrow s$ i $s_{1} \rightarrow s^{\prime }$ 

możemy funkcję wartościującą strategii wyrazić w sposób rekurencyjny:

$\qquad$$V^{\pi }(s) = E[R(s) + \gamma V^{\pi }(s^{\prime }) | s_{0}=s, \pi ]$

W tym wyrażeniu $s$ jest już ustalone więc 
* $E[R(s)] = R(s)$ 
natomiast 
* $s^{\prime }$ jest zmienną losową pochodzącą z rozkładu $P_{s, \pi (s)}$ więc 
$$E[V^{\pi }(s^{\prime }) ] = \sum _{s^{\prime }} P_{s, \pi (s)}(s^{\prime }) V^{\pi }(s^{\prime }) $$.

Zbierając razem te obserwacje i podstawiając je do równania otrzymujemy tzw. **równanie Bellmana**:

 $$V^{\pi }(s) = R(s) +\gamma \sum _{s^{\prime } \in S} P_{s,\pi (s)}(s^{\prime }) V^{\pi }(s^{\prime })$$

W szczególności dla MDP o skończonej liczbie stanów $N$ można napisać równanie Bellmana dla każdego stanu $s$. Otrzymamy wtedy układ $N$ równań liniowych z $N$ niewiadomymi ($V^{\pi }(s)$) , który można łatwo rozwiązać.
Popatrzmy na przykład z robotem.
Weźmy stan $(3,1)$ i strategię $\pi ((3,1)) = N$. Równanie Bellmana dla tego stanu to:

$$V^{\pi }\left( (3,1) \right) = R((3,1)) + \gamma [ 0.8 V^{\pi }((3,2)) +0.1 V^{\pi }((4,1)) +0.1 V^{\pi }((2,1)) ]$$

Dla każdego z 11 stanów można napisać takie równanie. Otrzymamy 11 równań zawierających 11 niewiadomych $V^{\pi }(i,j)$.

Zdefiniujmy optymalną funkcję wartościującą strategi jako:

$$
V^{*}(s) = \max _{\pi } V^{\pi }(s)
$$

Innymi słowy jest to największa możliwa suma nagród (ważonych przez $\gamma $) przy pomocy najlepszej strategii.

Można napisać równanie Bellmana dla $V^{*}(s)$:

$\qquad$$V^{*}(s) = R(s) +\max _{a}\gamma \sum _{s^{\prime } \in S} P_{s,a}(s^{\prime }) V^{*}(s^{\prime })$

* Pierwszy człon to natychmiastowa nagroda.
* W drugim członie suma wyraża wartość oczekiwaną zysku jeśli wybralibyśmy akcję $a$. Wartość ta jest maksymalizowana po $a$.

To prowadzi do definicji strategii $\pi ^{*}(s)$, czyli takiej strategii, która maksymalizuje zysk:

$$
\pi ^{*}(s) = \arg \max _{a} \sum _{s^{\prime }} P_{sa}(s^{\prime }) V^{*}(s^{\prime })
$$

W definicji tej nie musimy już uwzględniać $\gamma $ bo nie ma ona znaczenia dla operatora $\arg \max $. Taka definicja powoduje, że akcje podejmowane zgodnie ze strategią $\pi ^{*}$ prowadzą do maksymalizacji drugiego członu w równaniu Bellmana na $V^{*}$. W tym sensie strategia $\pi ^{*}$ jest strategią optymalną. Zauważmy, że $\pi ^{*}(s)$ jest strategią optymalną bez względu na to, z którego stanu uruchamiany jest proces.

## Algorytmy znajdowania optymalnej strategii:

Powyższe definicje nie dają niestety przepisu jak obliczyć $\pi ^{*}$. 

Właściwe algorytmy omówimy poniżej. 

## Algorytm iteracji funkcji wartościującej
Zauważmy, że gdybyśmy znali $V^{*}$ to można by je podstawić do ostatniego równania aby wyznaczyć $\pi ^{*}$.

Zatem pierwszy algorytm, który prowadzi do wyznaczenia optymalnej strategi to:
**iteracja funkcji wartościującej**:

1. inicjuj dla każdego $s$: $V(s) = 0$ (wypełniamy tablicę V zerami)

2. Powtarzaj aż zbiegniesz:
{

dla każdego stanu $s$
$$V(s) = R(s) + \max _{a} \gamma \sum _{s^{\prime }} P_{sa}(s^{\prime }) V(s^{\prime }) $$

}

Tak iterowane $V(s) \rightarrow V^{*}(s)$

* W algorytmie tym można zastosować uaktualnianie synchroniczne - wtedy najpierw obliczamy wszystkie prawe strony, wyniki obliczeń przechowując w zmiennej tymczasowej i dopiero po przebiegnięciu pętlą przez wszystkie stany następuje przypisanie uaktualniające.
* Druga możliwość to uaktualnianie asynchroniczne, tzn. w każdym kroku wybieramy jeden stan i uaktualniamy $V(s)$ dla tego stanu. W kolejnym kroku do obliczeń używamy już tego uaktualnionego $V$.
* Algorytm asynchroniczny jest zwykle nieco szybciej zbieżny niż synchroniczny. Po wyznaczeniu $V^{*}$ można go podstawić do równania i wyznaczyć optymalną strategię.

Stosując ten algorytm do naszego przykładu można otrzymać np. taka funkcję wartościującą $V^{*}$:


<table class="wikitable";style = font-size:100%>
	<tr>
		<td>0.86</td>
		<td>0.90</td>
		<td>0.93</td>
		<td>+1</td>
	</tr>
	<tr>
		<td>0.82</td>
		<td>•</td>
		<td>0.69</td>
		<td>-1</td>
	</tr>
	<tr>
		<td>0.78</td>
		<td>0.73</td>
		<td>0.71</td>
		<td>0.49</td>
	</tr>
</table>

Podstawiając do równania na $\pi ^{*}$ dostajemy, przykładowo dla stanu s= (3,1):

Dla akcji W:

$\qquad$$\sum _{s^{\prime }} P_{sa}(s^{\prime })V^{*}(s^{\prime }) = 0.8\cdot 0.73 + 0.1\cdot 0.69 + 0.1\cdot 0.71 = 0.724 $

Dla akcji N:

$\qquad$$\sum _{s^{\prime }} P_{sa}(s^{\prime })V^{*}(s^{\prime }) = 0.8\cdot 0.69 + 0.1\cdot 0.73 + 0.1\cdot 0.49 = 0.674 $

Dla akcji E:

$\qquad$$\sum _{s^{\prime }} P_{sa}(s^{\prime })V^{*}(s^{\prime }) = 0.8\cdot 0.49 + 0.1\cdot 0.69 + 0.1\cdot 0.71 = 0.532 $

Dla akcji S:

$\qquad$$\sum _{s^{\prime }} P_{sa}(s^{\prime })V^{*}(s^{\prime }) = 0.8\cdot 0.71 + 0.1\cdot 0.73 + 0.1\cdot 0.49 = 0.690 $

Zatem dla tego stanu wybrana byłaby akcja W. 

Analogiczne obliczenia trzeba przeprowadzić dla pozostałych stanów. Wyniki końcowe prezentuje poniższa tabela.


<table class="wikitable";style = font-size:100%>
	<tr>
		<td>E</td>
		<td>E</td>
		<td>E</td>
		<td>+1</td>
	</tr>
	<tr>
		<td>N</td>
		<td>•</td>
		<td>N</td>
		<td>-1</td>
	</tr>
	<tr>
		<td>N</td>
		<td>W</td>
		<td>W</td>
		<td>W</td>
	</tr>
</table>

### Algorytm iteracji strategii

Poniżej zaprezentowany jest algorytm iteracji strategii.

<ol>
	<li>
	inicjujemy $\pi $ w sposób losowy.
	<li>
	Powtarzaj, aż zbiegniesz:
{

rozwiąż równania Bellmana dla aktualnej strategii (dla procesów o bardzo dużej ilości stanów może to być nieefektywne):

$\qquad$$V = V^{\pi }$

Następnie oblicz lepsze $\pi $:

$\qquad$$\pi (s) = \arg \max _{a} \sum _{s^{\prime }}P_{sa}(s^{\prime }) V(s^{\prime })$

}

</ol>

Okazuje się, że powyższa iteracja daje $V \rightarrow V^{*}$ zaś $\pi \rightarrow \pi ^{*}$.

Dla małych i średnich problemów MDP iteracja strategii jest szybko zbieżna, ale dla dużych problemów staje się kosztowna obliczeniowo.

### Algorytm aktualizacji funkcji Q
Ostatni algorytm ma znamiona eksploracji środowiska. Potrzebna jest do tego funkcja jakości $Q$ (quality). Funkcja $Q$ zwraca jakość dla kombinacji stan-akcja:

$$Q: S \times A \to \mathbb{R}$$ 

Jakość ta mierzona jest przez oczekiwaną wartość całkowitej nagrody za wykonanie danej akcji $a$ w stanie $s$.

Czyli dla optymalnej funkcji $Q^*$ i $V^*$ zachodzi związek:
$$V^*(s) = \max_a Q^*(s,a) $$

Najlepszą strategią jest zatem wybieranie akcji  $\arg \max_a Q^*(s,a) $:
$$\forall_s \quad \pi^*(s) = \arg \max_a Q^*(s,a) $$

Funkcję $Q$ można znaleźć poprzez aktualizacje ważoną średnią dotychczasowej wartości i nowej informacji:
<b>iteracja funkcji Q</b>:

1. inicjuj dla każdego $s$: $Q(s,a) = 0$ (wypełniamy tablicę Q zerami)

2. Powtarzaj aż zbiegniesz:
{
* Robot ustawiany jest w losowej pozycji. 
* Przejdź epizod:
  * w każdej kolejnej chwili czasu  $t$ robot wybiera akcję  $a_t$, 
  * dostaje nagrodę $R_t$, 
  * przechodzi do nowego stanu $s_{t+1}$ ( to może zależeć od poprzedniego stanu  $s_t$ i wybranej akcji), 
  * $Q$ jest uaktualniane:
$$Q(s_{t},a_{t}) := (1-\alpha) \cdot \underbrace{Q(s_{t},a_{t})}_{\textrm{ stara wartość}} + \alpha \cdot  \overbrace{\bigg( R_{t} +{\gamma} \cdot \underbrace{\max_{a}Q(s_{t+1}, a)}_{\textrm{ estymata optymalnej przyszłej wartości}} \bigg) }^{\textrm{nowa informacja o wartości pary } (s_t,a_t)}$$ 
  * gdzie ''$R_{t}$'' nagroda w bieżącym stanie $s_t$, i $\alpha$ jest prędkością uczenia ($0 < \alpha \le 1$).
  * Epizod nauki kończy się gdy $s_{t+1}$ jest stanem końcowym.

Przykład: https://drive.google.com/file/d/18j357je8pSXi3SOYZgK1qp1KWrMsEc2Z/view?usp=sharing

## Estymowanie modelu dla MDP

Na początku powiedzieliśmy, że MDP jest określony przez podanie następującej krotki: $(S,A,\lbrace P_{sa}\rbrace ,\gamma ,R)$.

* Stany i możliwe akcje zwykle wynikają w sposób naturalny z rozważanego problemu. 
* $\gamma $ i $R$ wybieramy sami. 
* Najtrudniejsze do wymyślenia <i>ad hoc</i> są prawdopodobieństwa przejść. 
  * Można jednak wyestymować je z danych (przez obserwację wielu realizacji procesu). Używamy wtedy estymatora największej wiarygodności, tzn:

$$P_{sa}(s^{\prime }) = \frac{\#\text{ kiedy wybrano akcje }a\text{ w stanie }s\text{ i otrzymano stan }s^{\prime }}{\#\text{ kiedy wybrano akcję }a\text{ w stanie }s}$$

Zatem dla MDP o nieznanych prawdopodobieństwach przejść można zastosować następujący algorytm:

* inicjalizuj $\pi $ losowo
* powtarzaj aż zbiegniesz:
{
  *	Wykonaj strategię $\pi $ w MDP dla pewnej liczby prób.
  *	Na podstawie zaobserwowanych sekwencji stanów estymuj $P_{sa}$
  *	zastosuj algorytm iteracji funkcji wartościującej aby oszacować $V$
  *	Uaktualnij $\pi $
}

W powyższym zastosowaniu algorytm iteracji funkcji wartościującej można zmodyfikować tak, aby nie zaczynał on za każdym razem inicjalizacji od $V(s) = 0$, ale od $V$ które uzyskano w poprzednim kroku zewnętrznej pętli.

# Prośba o wypełnienie ankiety